In [ ]:
from urllib import request
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import datetime
import time
import pandas as pd
from math import ceil
import os

In [ ]:
url = 'https://www.airkorea.or.kr/autoStatistic'
browser = webdriver.Chrome('C:/0.bigData/0.sw/chromedriver.exe')
browser.get(url)

In [ ]:
dosi_list = ['서울', '경기', '인천', '강원', '충남', '대전', '충북', '부산',
             '울산', '대구', '경북', '경남', '전남', '광주', '전북', '제주']
item_list = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM2.5']

dosi_select = browser.find_element_by_id('district')
item_select = browser.find_element_by_id('item_code')
from_date = browser.find_element_by_id('from_date')
to_date = browser.find_element_by_id('to_date')
check_list = browser.find_element_by_css_selector('#ddcl-station_list > span > span')

search = browser.find_element_by_css_selector('#cont_body > div.search_head.mb20 > div:nth-child(2) > input[type="image"]:nth-child(7)')

In [ ]:
def week_of_month(date):
    first_day_week = (date.replace(day=1).weekday() + 1) % 7
    return (date.day + first_day_week - 1) // 7 + 1


def generate_date(start, end):
    date = start
    while date <= end:
        yield date
        date += datetime.timedelta(1) 

        
def click_date(date):
    year = str(date.year)
    month = str(date.month-1)
    isoweekday = week_of_month(date)
    weekday = ((date.weekday() + 1) % 7) + 1 
    print(isoweekday, weekday)
    from_date.click()
    Select(browser.find_element_by_class_name('ui-datepicker-year')).select_by_value(year)
    Select(browser.find_element_by_class_name('ui-datepicker-month')).select_by_value(month)
    f = browser.find_element_by_css_selector(f'#ui-datepicker-div > table > tbody > tr:nth-child({isoweekday}) > td:nth-child({weekday}) > a').click()
    to_date.click()
    Select(browser.find_element_by_class_name('ui-datepicker-year')).select_by_value(year)
    Select(browser.find_element_by_class_name('ui-datepicker-month')).select_by_value(month)
    browser.find_element_by_css_selector(f'#ui-datepicker-div > table > tbody > tr:nth-child({isoweekday}) > td:nth-child({weekday}) > a').click()
    

def click_gu(click_index):
    check_list = browser.find_element_by_css_selector('#ddcl-station_list > span > span')
    check_list.click()
    if click_index != 1:
        pre_click_index = click_index - 1
        pre_click = browser.find_element_by_id(f'ddcl-station_list-i{pre_click_index}')
        pre_click.click()
    click = browser.find_element_by_id(f'ddcl-station_list-i{click_index}')
    click.click()
    check_list.click()
        
def get_gu_list():
    check_list = browser.find_element_by_css_selector('#ddcl-station_list > span > span')
    check_list.click()
    tag_list = browser.find_elements_by_class_name('ui-dropdownchecklist-text')
    gu_list = [tag.text for tag in tag_list[2:]]
    check_list.click()
    return gu_list

In [ ]:
start = datetime.date(2017,1,25)
end = datetime.date(2018,3,28)

In [ ]:
for dosi in dosi_list:
    dosi_select.send_keys(dosi)
    if not os.path.isdir(dosi):
        os.mkdir(dosi)
    for i, gu in enumerate(get_gu_list(), 1):
        if not os.path.isdir(os.path.join(dosi, gu)):
            os.mkdir(os.path.join(dosi, gu))
        click_gu(i)
        df = pd.DataFrame(columns=item_list)
        for date in generate_date(start, end):
            click_date(date)
            for item in item_list:
                item_select.send_keys(item)
                search.click()
                time.sleep(1)
                a = browser.find_elements_by_css_selector('#tablefix1 > tbody > tr')
                if len(a) < 100:
                    continue
                df[item] = [browser.find_elements_by_css_selector('#tablefix1 > tbody > tr:nth-child({}) > td:nth-child(2)'.format(i))[-1].text for i in range(3, 27)]
                
            df.to_csv('{}.csv'.format(os.path.join(dosi, gu, date.strftime('%Y%m%d'))))